# Introduction to TensorFlow
---

In [1]:
import tensorflow as tf 

# In Tensorflow, all values are encapsulated in a tensor object
hello_constant = tf.constant('Hello World!')            # # 0-dim string tensor
A = tf.constant(1234)                                   # 0-dim int32 tensor
B = tf.constant([123, 456, 789])                        # 1-dim int32 tensor
C = tf.constant([[123, 456, 789], [222,333,444]])       # 2-dim int32 tensor

# TensorFlow's API is built around a computational graph
# It is an environment for running a graph that is in charge of allocating GPU and/or CPU
with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

C:\Users\dtr\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


b'Hello World!'


### Input: tf.placeholder() --- (has to go with Session's feed_dict)
Allow to set input right before the session runs

*** Note that tf.placeholder() and tf.constant() can't be modified

In [9]:
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello World', y: 123, z: 45.67})
    print(output)

Hello World


### Math

In [13]:
x_add = tf.add(5, 2)    # 7
x_subtract = tf.subtract(10, 4)   # 6
x_multiply = tf.multiply(2, 5)   # 10
x_convertType = tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))

with tf.Session() as sess:
    output = sess.run(x_convertType)
    print(output)

1


### tf.Variable() --- (can be modified => for training NN)

In [ ]:
x = tf.Variable(5)

# Initialize weights with random numbers from a normal distribution => good practice
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

# Since the weights are already helping prevent the model from getting stuck,
# don't need to randomize the bias
bias = tf.Variable(tf.zeros(n_labels))

# Need to initialize the state of all the Variable tensors
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

In [ ]:
# Example of Softmax
def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)
    
    # TODO: Calculate the softmax of the logits
    softmax = tf.nn.softmax(logits)    
    
    with tf.Session() as sess:
        pass
        # TODO: Feed in the logit data
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output

## MNIST Tutorial

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
from helper import batches  # Helper function created in Mini-batching section


def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 10
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))